In [1]:
import torch
import os
import numpy as np
import datetime

from numpy import genfromtxt
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn import RNN, LSTM, GRU
from torcheval.metrics import MulticlassAccuracy
from ray import tune
from ray.train import Checkpoint
from ray.air import session
from ray.tune.schedulers import ASHAScheduler
from functools import partial

from model import *
from dataset import MyDataset


from utils import read_data

In [2]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
device = torch.device('cpu')

In [3]:
train_path = "../nn4_data/UCI HAR Dataset/train/Inertial Signals/"



In [4]:
def read_data(data_path):
    files = os.listdir(data_path)
    file_path = data_path + files[0]
    train_data = torch.tensor(genfromtxt(file_path)).unsqueeze(2)
    for file in os.listdir(train_path)[1:]:
        file_path = data_path + file
        new_vector = torch.tensor(genfromtxt(file_path)).unsqueeze(2)
        train_data = torch.concat((train_data, new_vector), 2)

    return train_data.to(dtype=torch.float32)

    

In [5]:
x_data = read_data(train_path).to(device=device)
print(x_data.shape)
y_path = "../nn4_data/UCI HAR Dataset/train/y_train.txt"
y_data = (torch.tensor(genfromtxt(y_path)) - 1).to(device=device)

full_dataset = MyDataset(x_data, y_data)
train_dataset, validation_dataset = random_split(full_dataset, (0.8, 0.2))
train_loader, validation_loader = DataLoader(train_dataset, batch_size=8), DataLoader(validation_dataset, batch_size=8)

torch.Size([7352, 128, 9])


In [6]:
def training_loop(n_epochs, optimizer, scheduler, model, loss_fn, train_loader, validation_loader, ):

    best_loss = None
    counter = 0
    for epoch in range(n_epochs):
        for data, label in train_loader:
            hidden = model.init_hidden().to(device=device)
            output, hidden = model(data, hidden)
            loss = loss_fn(output, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        scheduler.step()
        summary_loss = 0
        for data, label in validation_loader:
            hidden = model.init_hidden().to(device=device)        
            output, hidden = model(data, hidden)
            summary_loss += loss_fn(output, label)

        validation_loss = summary_loss / len(validation_loader)

        if best_loss and validation_loss > best_loss:
            counter += 1
            if counter == 5:
                print(f"Early stop on epoch {epoch}")
                break
        else:
            counter = 0
            best_loss = validation_loss

        if epoch == 1 or epoch % 5 == 0:

            print('{} Epoch {}, Training loss {}, Validation loss {}, lr {}'.format(
                datetime.datetime.now(),
                epoch,
                loss / len(train_loader),
                validation_loss,
                scheduler.get_last_lr())
            )

In [7]:
torch.version.cuda
print(torch.__version__)

2.2.2+cu121


In [8]:
input_size = 9
num_layers = 1
hidden_size = 6


rnn_models = {"classic RNN": SimpleRNN(input_size, num_layers, hidden_size),
              "bidirectional RNN": BidirectionalRNN(input_size, num_layers, hidden_size),
              "LSTM RNN": LSTMRNN(input_size, num_layers, hidden_size),
              "GRU RNN": GRURNN(input_size, num_layers, hidden_size),
              }

for name, model in rnn_models.items():
    model = model.to(device=device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=30)

    loss_fn = torch.nn.CrossEntropyLoss()
    training_loop(100, optimizer=optimizer, scheduler=lr_scheduler, model=model, loss_fn=loss_fn, train_loader=train_loader, validation_loader=validation_loader)
    torch.save(model.state_dict(), f"{name}.pt")

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


2024-04-05 01:03:43.867007 Epoch 0, Training loss 0.002523114439100027, Validation loss 1.7332574129104614, lr [0.001]
2024-04-05 01:04:37.606964 Epoch 1, Training loss 0.0025311585050076246, Validation loss 1.7103959321975708, lr [0.001]
2024-04-05 01:08:07.637600 Epoch 5, Training loss 0.002430521184578538, Validation loss 1.6840683221817017, lr [0.001]
2024-04-05 01:12:29.915653 Epoch 10, Training loss 0.002282110508531332, Validation loss 1.660799264907837, lr [0.001]
2024-04-05 01:16:55.646888 Epoch 15, Training loss 0.0022301622666418552, Validation loss 1.6460583209991455, lr [0.001]
2024-04-05 01:21:30.931399 Epoch 20, Training loss 0.0021963517647236586, Validation loss 1.6361205577850342, lr [0.001]
2024-04-05 01:26:07.401950 Epoch 25, Training loss 0.0021739790681749582, Validation loss 1.6288881301879883, lr [0.001]
2024-04-05 01:30:42.214079 Epoch 30, Training loss 0.0021605573128908873, Validation loss 1.62420654296875, lr [0.0001]
2024-04-05 01:35:17.513530 Epoch 35, Tra